# Churn Rate Analysis
Esse notebook contempla a análise exploratória sobre a retenção de profissionais de medicina e enfermagem nas regiões de saúde com base em dados do CNES-PF.

## Carregar bibliotecas

In [ ]:
import numpy as np
# Reload automático no Jupyter Notebook
%reload_ext autoreload
%autoreload 2

import traceback
import warnings
from dados import *
from tratamentos import *
from modelagem import *
from visualizacoes import *
from avaliacao import *

## Configurar bibliotecas

In [ ]:
# Remover limite de exibição de linhas e colunas do pandas
pd.set_option('max_rows', 9999)
pd.set_option('max_columns', 9999)

# Evitar notações científicas no pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Evitar warnings sobre perfomance no pandas
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

## Obter dados

In [ ]:
df = obter_dados('profissionais.sql')
df.head()

In [ ]:
geo = obter_dados('municipios.sql')
geo.head()

In [ ]:
# Realizar o join dos dados do CNES com tabela de municipios
df = df.merge(geo, left_on=['CODUFMUN'], right_on=['cod_municipio'])
df.head()

## Processar os dados e criar os gráficos e arquivos CSV de retenção por região de saúde

In [ ]:
# Realizar processamento para cada categoria profissional
for categoria in ['Médico']:
#for categoria in df['categoria'].unique():
    df_categoria = df.loc[df['categoria'] == categoria].copy()

    # Criar listas para salvar dados a serem exportados
    estatisticas_curvas = []
    triangulos = []
    mapes = []
    retencao_ano = []
    retencao_geral = []

    # Realizar o processamento separado para cada região de saúde
    i=0
    #for regiao_saude in ['11007']:
    for regiao_saude in df['cod_regsaud'].unique():
        try:
            print(f'Processando {regiao_saude} - {i+1}/{len(df["cod_regsaud"].unique())}')
            df_categoria_regiao = df_categoria.loc[df_categoria['cod_regsaud'] == regiao_saude].copy()

            # Agregar os dados por cohorts
            cohort_pivot, cohort_sizes = gerar_triangulo_agregado(df_categoria_regiao.copy())

            for fill_na in list(range(0, 181, 12)):
                if fill_na not in cohort_pivot.columns:
                    cohort_pivot[fill_na] = np.nan

            # Obter o modelo chainladder
            cl_model = treinar_chainladder(cohort_pivot.copy())

            # Calcular profissionais restantes em cada período do cohort
            restantes = calcular_restantes(cl_model, cohort_pivot.copy())

            # Realizar a divisão de cada célula da tabela pivotada pelo tamanho do grupo para deixar em percentual
            percentual_restantes = calcular_percentual_restantes(restantes, cohort_sizes)

            # Calcular retenção anual (últimos 5 anos) por região de saúde
            retencao_anual = calcular_retencao_ano_regiao(percentual_restantes.copy(), regiao_saude)
            retencao_ano.append(retencao_anual)

            # Calcular retenção geral (média de todo o triangulo)
            retencao_geral.append(pd.DataFrame({'regiao_saude': [regiao_saude], 'retencao_geral': [cohort_pivot.divide(cohort_sizes, axis = 0).unstack().mean()]}))

            # Salvar dados do triangulo para exportar em csv
            cohort_pivot['cod_regsaud'] = regiao_saude
            triangulos.append(cohort_pivot.copy())
            cohort_pivot.drop(['cod_regsaud'], axis=1, inplace=True)

            # Criar features para ML sobre caracteristicas estatisticas da curva
            obs = gerar_ml_features(cohort_pivot, regiao_saude)
            estatisticas_curvas.append(obs)

            # Calcular mape
            mape = calcular_mape(cl_model).stack().mean()
            mapes.append(pd.DataFrame({'cod_regsaud': [regiao_saude], 'mape': [mape]}))

            # Gerar gráficos
            nome_regiao = df_categoria_regiao["regiao_saude"].values[0]
            uf_regiao = df_categoria_regiao["uf_sigla"].values[0]
            gerar_graficos(restantes.copy(), percentual_restantes.copy(), cl_model, f'Região de Saúde {nome_regiao} - {uf_regiao}', f'{categoria}{regiao_saude}')

            i+=1
        except Exception as e:
            print(f'Erro na região de saúde {regiao_saude}: {e}')
            print(traceback.format_exc())
            continue


    # Exportar dados em CSV
    pd.concat(estatisticas_curvas).to_csv(f'csvs/{categoria}_estatisticas_curvas.csv', sep=';')
    pd.concat(triangulos).to_csv(f'csvs/{categoria}_triangulos.csv', sep=';')
    pd.concat(mapes).to_csv(f'csvs/{categoria}_mapes.csv', sep=';')
    pd.concat(retencao_ano).to_csv(f'csvs/{categoria}_retencao_ano.csv', sep=';')
    pd.concat(retencao_geral).to_csv(f'csvs/{categoria}_retencao_geral.csv', sep=';')